In [1]:
import sheffkeys,tweepy,time
auth=tweepy.OAuthHandler(sheffkeys.consumer_key,sheffkeys.consumer_secret)
auth.set_access_token(sheffkeys.access_token,sheffkeys.access_token_secret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
from pymongo import MongoClient
atlas_client = MongoClient(sheffkeys.mongo_connection_string)
dbase = atlas_client.HurricaneZeta
import json
import tweepy

class TweetListener(tweepy.StreamListener):
    
    """Handles incoming Tweet stream."""

    def __init__(self, api, database, limit=10000000):
            """Create instance variables for tracking number of tweets."""
            self.dbase = database
            self.tweet_count = 0
            self.TWEET_LIMIT = limit  # 10,000 by default
            super().__init__(api)  # call superclass's init

    def on_connect(self):
        """Called when your connection attempt is successful, enabling 
       you to perform appropriate application tasks at that point.
       """
    print('Successfully connected to Twitter\n')

    def on_data(self, data):
        """Called when Twitter pushes a new tweet to you."""
        self.tweet_count += 1  # track number of tweets processed
        json_data = json.loads(data)  # convert string to JSON
        self.dbase.tweets.insert_one(json_data)  # store in tweets collection
        print(f'    Screen name: {json_data["user"]["name"]}') 
        print(f'     Created at: {json_data["created_at"]}') 
        print(f'Tweets received: {self.tweet_count}')  

       # if TWEET_LIMIT is reached, return False to terminate streaming
        return self.tweet_count != self.TWEET_LIMIT
    def on_error(self, status):
        print(status)
        return True
tweet_limit=10000000
twitter_stream=tweepy.Stream(api.auth,TweetListener(api,dbase,tweet_limit))
twitter_stream.filter(track=['#hurricanezeta' or 'hurricanezeta' or '#zeta'],languages=["en"],is_async=True)

Successfully connected to Twitter

